In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import cv2 as cv
from PIL import Image

In [ ]:
%matplotlib inline

In [ ]:
%cd '/content/drive/MyDrive/Datasets/Malaria/cell_images/'

In [ ]:
un="Uninfected"
print("no of uninfected images : ",len(os.listdir(un)))

In [ ]:
para="Parasitized"
print("No parasitized images : ",len(os.listdir(para)))

In [ ]:
for image in tqdm(os.listdir(un)):
    if image!="Thumbs.db":
        img=cv.imread(os.path.join(un,image))
        print(img.shape)
        break

In [ ]:
unin=[]
for image in tqdm(os.listdir(un)):
    if image!="Thumbs.db":
        img=Image.open(os.path.join(un,image))
        img=img.resize((64,64))
        img=np.asarray(img)
        unin.append(img)

In [ ]:
print(len(unin[794]))
print(len(unin[7]))

In [ ]:
par=[]
for image in tqdm(os.listdir(para)):
    if image!="Thumbs.db":
        img=Image.open(os.path.join(para,image))
        img=img.resize((64,64))
        img=np.asarray(img)
        par.append(img)

In [ ]:
print(len(par))

In [ ]:
ov_images=unin+par

In [ ]:
print(len(ov_images))

In [ ]:
labels=([0]*(len(os.listdir(un))-1))+([1]*(len(os.listdir(para))-1))

In [ ]:
print(len(labels))

In [ ]:
ov_images=np.array(ov_images)

In [ ]:
labels=np.array(labels)

In [ ]:
ov_images=ov_images/255

In [ ]:
from sklearn.utils import shuffle

In [ ]:
ov_images,labels=shuffle(ov_images,labels)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(ov_images,labels,test_size=0.2)

In [ ]:
print(x_train.shape,y_train.shape)

In [ ]:
print(x_test.shape,y_test.shape)

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
model1=keras.applicaitons.xception.Xception(
    include_top=False,
    weights="imagenet",
    pooling="avg"
)

In [ ]:
for layer in model.layers:
    layer.trainable = False

class Mish(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    x=x*tensorflow.math.tanh(F.softplus(x))
    return x

model2 = tf.keras.Sequential([
        layers.Conv2D(filters=32, kernel_size=(3,3),input_shape=(64,64,3),activation='relu',padding="same")),
        layers.MaxPooling2D(pool_size=(2,2),strides=2)),
        layers.Conv2D(filters=64, kernel_size=(3,3),input_shape=(64,64,3),activation='relu',padding="same"),
        layers.BatchNormalization(axis=-1),
        layers.Activation(Mish()),
        layers.MaxPooling2D(pool_size=(2,2),strides=2),
        layers.Conv2D(filters=128, kernel_size=(3,3),input_shape=(64,64,3),activation='relu',padding="same"),
        layers.MaxPooling2D(pool_size=(2,2),strides=2),
        layers.Dropout(rate=0.2),
        layers.Dense(128,activation='relu'),
        layers.Flatten(),
        layers.Dense(50,activation='relu'),
        layers.Dropout(rate=0.2),
        layers.Dense(1,activation='softmax'),
])

model3=Sequential()
model3.add(model1)
model3.add(model2)
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model3.summary()

In [ ]:
history=model3.fit(x_train,y_train,verbose=1,epochs=1,validation_data=(x_test,y_test), shuffle=True)

In [ ]:
yhat=model.predict(x_test)>0.5

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix,f1_score

In [ ]:
f1_score(y_test,yhat)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix=confusion_matrix(yhat,y_test)
print(cf_matrix)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Purples')